# Apache Hudi Core Conceptions (4) - MOR: Compaction

## 1. Configuration

In [1]:
%%sh
# deploy hudi bundle jar
hdfs dfs -copyFromLocal -f /usr/lib/hudi/hudi-spark-bundle.jar /tmp/hudi-spark-bundle.jar
hdfs dfs -ls /tmp/hudi-spark-bundle.jar

-rw-r--r--   1 emr-notebook hdfsadmingroup   61421977 2023-03-22 01:43 /tmp/hudi-spark-bundle.jar


In [2]:
%%configure -f
{
    "conf" : {
        "spark.jars":"hdfs:///tmp/hudi-spark-bundle.jar",            
        "spark.serializer":"org.apache.spark.serializer.KryoSerializer",
        "spark.sql.extensions":"org.apache.spark.sql.hudi.HoodieSparkSessionExtension",
        "spark.sql.catalog.spark_catalog":"org.apache.spark.sql.hudi.catalog.HoodieCatalog"
    }
}

In [3]:
%env S3_BUCKET=apache-hudi-core-conceptions

env: S3_BUCKET=apache-hudi-core-conceptions


In [4]:
%%sql
set S3_BUCKET=apache-hudi-core-conceptions

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
106,application_1678096020253_0216,spark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [5]:
%env WORKSPACE=/home/emr-notebook/apache-hudi-core-conceptions

env: WORKSPACE=/home/emr-notebook/apache-hudi-core-conceptions


In [6]:
%%sh
# make workspace
mkdir -p $WORKSPACE
# deploy hudi-stat.sh, a utility shell script to output hudi table status
wget https://raw.githubusercontent.com/bluishglc/apache-hudi-core-conceptions/master/hudi-stat.sh -O $WORKSPACE/hudi-stat.sh &>/dev/null
chmod a+x $WORKSPACE/hudi-stat.sh
ls $WORKSPACE/hudi-stat.sh

/home/emr-notebook/apache-hudi-core-conceptions/hudi-stat.sh


In [7]:
%%html
<style>
table {float:left}
</style>

## 2. Test Case 1 - Sync Compaction ( Inline Schedule + Inline Execute )

### 2.1. Test Plan

Step No.|Action|Volume Per Partition |Storage
:--------:|:------|:------|:----------
1|Insert|96MB|+1 Base File
2|Update|788KB|+1 Log File
3|Update|1.2MB|+1 Log File +1 Compacted Base File

### 2.2. Key Settings

KEY|DEFAULT VALUE|SET VALUE
:---|:---|:---
hoodie.compact.inline|false|true
hoodie.compact.schedule.inline|false|false
hoodie.compact.inline.max.delta.commits|5|3
hoodie.copyonwrite.record.size.estimate|1024|175

### 2.3. Set Variables

In [8]:
%%sql
set TABLE_NAME=reviews_mor_compaction_1

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [9]:
%env TABLE_NAME=reviews_mor_compaction_1

env: TABLE_NAME=reviews_mor_compaction_1


### 2.4. Create Table

In [10]:
%%sh
aws s3 rm s3://${S3_BUCKET}/${TABLE_NAME} --recursive &>/dev/null
rm -rf ${WORKSPACE}/${TABLE_NAME}
sleep 5

In [11]:
%%sql
drop table if exists ${TABLE_NAME}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [12]:
%%sql
drop table if exists ${TABLE_NAME}_ro

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [13]:
%%sql
drop table if exists ${TABLE_NAME}_rt

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [14]:
%%sql
create table if not exists ${TABLE_NAME} (
    review_id string, 
    star_rating long, 
    review_body string, 
    review_date date, 
    year long,
    timestamp long,
    parity int
)
using hudi
location 's3://${S3_BUCKET}/${TABLE_NAME}'
partitioned by (parity)
options ( 
    type = 'mor',  
    primaryKey = 'review_id', 
    preCombineField = 'timestamp',
    hoodie.compact.inline = 'true',
    hoodie.compact.schedule.inline = 'false',
    hoodie.compact.inline.max.delta.commits = '3',
    hoodie.copyonwrite.record.size.estimate = '175'
);

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

### 2.5. Insert 96MB / +1 Base File

In [15]:
%%sql
insert into 
    ${TABLE_NAME}
select 
    review_id, 
    star_rating, 
    review_body, 
    review_date, 
    year,
    unix_timestamp(current_timestamp()) as timestamp,
    mod(crc32(review_id), 2) as parity
from
    reviews
where
    year = 2003

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [16]:
%%sh
${WORKSPACE}/hudi-stat.sh s3://${S3_BUCKET}/${TABLE_NAME} timeline commits compactions storage


[ TIMELINE ]

╔═════╤═══════════════════╤═════════════╤═══════════╤═════════════╤═════════════╤═════════════╗
║ No. │ Instant           │ Action      │ State     │ Requested   │ Inflight    │ Completed   ║
║     │                   │             │           │ Time        │ Time        │ Time        ║
╠═════╪═══════════════════╪═════════════╪═══════════╪═════════════╪═════════════╪═════════════╣
║ 0   │ 20230322014432551 │ deltacommit │ COMPLETED │ 03-22 01:44 │ 03-22 01:44 │ 03-22 01:45 ║
╚═════╧═══════════════════╧═════════════╧═══════════╧═════════════╧═════════════╧═════════════╝

[ COMMITS ]

╔═══════════════════╤═════════════════════╤═══════════════════╤═════════════════════╤══════════════════════════╤═══════════════════════╤══════════════════════════════╤══════════════╗
║ CommitTime        │ Total Bytes Written │ Total Files Added │ Total Files Updated │ Total Partitions Written │ Total Records Written │ Total Update Records Written │ Total Errors ║
╠═══════════════════╪════════

### 2.6. Update 787KB / +1 Log File

In [17]:
%%sql
update
    ${TABLE_NAME}
set             
    review_body = concat(uuid(),uuid(),uuid(),uuid(),uuid(),uuid(),uuid(),uuid(),uuid(),uuid()),
    timestamp = unix_timestamp(current_timestamp())
where
    review_date = '2003-01-01'
;

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [18]:
%%sh
${WORKSPACE}/hudi-stat.sh s3://${S3_BUCKET}/${TABLE_NAME} timeline commits compactions storage


[ TIMELINE ]

╔═════╤═══════════════════╤═════════════╤═══════════╤═════════════╤═════════════╤═════════════╗
║ No. │ Instant           │ Action      │ State     │ Requested   │ Inflight    │ Completed   ║
║     │                   │             │           │ Time        │ Time        │ Time        ║
╠═════╪═══════════════════╪═════════════╪═══════════╪═════════════╪═════════════╪═════════════╣
║ 0   │ 20230322014432551 │ deltacommit │ COMPLETED │ 03-22 01:44 │ 03-22 01:44 │ 03-22 01:45 ║
╟─────┼───────────────────┼─────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 1   │ 20230322014557951 │ deltacommit │ COMPLETED │ 03-22 01:46 │ 03-22 01:46 │ 03-22 01:46 ║
╚═════╧═══════════════════╧═════════════╧═══════════╧═════════════╧═════════════╧═════════════╝

[ COMMITS ]

╔═══════════════════╤═════════════════════╤═══════════════════╤═════════════════════╤══════════════════════════╤═══════════════════════╤══════════════════════════════╤══════════════╗
║ CommitTime        

### 2.7. Update 1.2MB / +1 Log File +1 Base File

In [19]:
%%sql
update
    ${TABLE_NAME}
set             
    review_body = concat(uuid(),uuid(),uuid(),uuid(),uuid(),uuid(),uuid(),uuid(),uuid(),uuid()),
    timestamp = unix_timestamp(current_timestamp())
where
    review_date = '2003-01-02'
;

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [20]:
%%sh
${WORKSPACE}/hudi-stat.sh s3://${S3_BUCKET}/${TABLE_NAME} timeline commits compactions storage


[ TIMELINE ]

╔═════╤═══════════════════╤═════════════╤═══════════╤═════════════╤═════════════╤═════════════╗
║ No. │ Instant           │ Action      │ State     │ Requested   │ Inflight    │ Completed   ║
║     │                   │             │           │ Time        │ Time        │ Time        ║
╠═════╪═══════════════════╪═════════════╪═══════════╪═════════════╪═════════════╪═════════════╣
║ 0   │ 20230322014432551 │ deltacommit │ COMPLETED │ 03-22 01:44 │ 03-22 01:44 │ 03-22 01:45 ║
╟─────┼───────────────────┼─────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 1   │ 20230322014557951 │ deltacommit │ COMPLETED │ 03-22 01:46 │ 03-22 01:46 │ 03-22 01:46 ║
╟─────┼───────────────────┼─────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 2   │ 20230322014638169 │ deltacommit │ COMPLETED │ 03-22 01:46 │ 03-22 01:46 │ 03-22 01:46 ║
╟─────┼───────────────────┼─────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 3   │ 20230322014653216

## 3. Test Case 2 - Async Compaction ( Offline Schedule, Offline Execute )

### 3.1. Test Plan

Step No.|Action|Volume Per Partition |Storage
:--------:|:------|:------|:----------
1|Insert|96MB|+1 Base File
2|Update|788KB|+1 Log File
3|Update|1.2MB|+1 Log File
4|Offline Schedule|N/A|N/A
5|Offline Execute|96.15MB|+1 Compacted Base File

### 3.2. Key Settings

KEY|DEFAULT VALUE|SET VALUE
:---|:---|:---
hoodie.compact.inline|false|true
hoodie.compact.schedule.inline|false|false
hoodie.compact.inline.max.delta.commits|5|3
hoodie.copyonwrite.record.size.estimate|1024|175

### 3.3. Set Variables

In [21]:
%%sql
set TABLE_NAME=reviews_mor_compaction_2

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [22]:
%env TABLE_NAME=reviews_mor_compaction_2

env: TABLE_NAME=reviews_mor_compaction_2


### 3.4. Create Table

In [23]:
%%sh
aws s3 rm s3://${S3_BUCKET}/${TABLE_NAME} --recursive &>/dev/null
rm -rf ${WORKSPACE}/${TABLE_NAME}
sleep 5

In [24]:
%%sql
drop table if exists ${TABLE_NAME}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [25]:
%%sql
drop table if exists ${TABLE_NAME}_ro

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [26]:
%%sql
drop table if exists ${TABLE_NAME}_rt

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [27]:
%%sql
create table if not exists ${TABLE_NAME} (
    review_id string, 
    star_rating long, 
    review_body string, 
    review_date date, 
    year long,
    timestamp long,
    parity int
)
using hudi
location 's3://${S3_BUCKET}/${TABLE_NAME}'
partitioned by (parity)
options ( 
    type = 'mor',  
    primaryKey = 'review_id', 
    preCombineField = 'timestamp',
    hoodie.compact.inline = 'false',
    hoodie.compact.schedule.inline = 'false',
    hoodie.compact.inline.max.delta.commits = '3',
    hoodie.copyonwrite.record.size.estimate = '175'
);

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

### 3.5. Insert 96MB / +1 Base File

In [28]:
%%sql
insert into 
    ${TABLE_NAME}
select 
    review_id, 
    star_rating, 
    review_body, 
    review_date, 
    year,
    unix_timestamp(current_timestamp()) as timestamp,
    mod(crc32(review_id), 2) as parity
from
    reviews
where
    year = 2003

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [29]:
%%sh
${WORKSPACE}/hudi-stat.sh s3://${S3_BUCKET}/${TABLE_NAME} timeline commits compactions storage


[ TIMELINE ]

╔═════╤═══════════════════╤═════════════╤═══════════╤═════════════╤═════════════╤═════════════╗
║ No. │ Instant           │ Action      │ State     │ Requested   │ Inflight    │ Completed   ║
║     │                   │             │           │ Time        │ Time        │ Time        ║
╠═════╪═══════════════════╪═════════════╪═══════════╪═════════════╪═════════════╪═════════════╣
║ 0   │ 20230322014803020 │ deltacommit │ COMPLETED │ 03-22 01:48 │ 03-22 01:48 │ 03-22 01:48 ║
╚═════╧═══════════════════╧═════════════╧═══════════╧═════════════╧═════════════╧═════════════╝

[ COMMITS ]

╔═══════════════════╤═════════════════════╤═══════════════════╤═════════════════════╤══════════════════════════╤═══════════════════════╤══════════════════════════════╤══════════════╗
║ CommitTime        │ Total Bytes Written │ Total Files Added │ Total Files Updated │ Total Partitions Written │ Total Records Written │ Total Update Records Written │ Total Errors ║
╠═══════════════════╪════════

### 3.6. Update 787KB / +1 Log File

In [30]:
%%sql
update
    ${TABLE_NAME}
set             
    review_body = concat(uuid(),uuid(),uuid(),uuid(),uuid(),uuid(),uuid(),uuid(),uuid(),uuid()),
    timestamp = unix_timestamp(current_timestamp())
where
    review_date = '2003-01-01'
;

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [31]:
%%sh
${WORKSPACE}/hudi-stat.sh s3://${S3_BUCKET}/${TABLE_NAME} timeline commits compactions storage


[ TIMELINE ]

╔═════╤═══════════════════╤═════════════╤═══════════╤═════════════╤═════════════╤═════════════╗
║ No. │ Instant           │ Action      │ State     │ Requested   │ Inflight    │ Completed   ║
║     │                   │             │           │ Time        │ Time        │ Time        ║
╠═════╪═══════════════════╪═════════════╪═══════════╪═════════════╪═════════════╪═════════════╣
║ 0   │ 20230322014803020 │ deltacommit │ COMPLETED │ 03-22 01:48 │ 03-22 01:48 │ 03-22 01:48 ║
╟─────┼───────────────────┼─────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 1   │ 20230322014910700 │ deltacommit │ COMPLETED │ 03-22 01:49 │ 03-22 01:49 │ 03-22 01:49 ║
╚═════╧═══════════════════╧═════════════╧═══════════╧═════════════╧═════════════╧═════════════╝

[ COMMITS ]

╔═══════════════════╤═════════════════════╤═══════════════════╤═════════════════════╤══════════════════════════╤═══════════════════════╤══════════════════════════════╤══════════════╗
║ CommitTime        

### 3.7. Update 1.2MB / +1 Log File +1 Base File

In [32]:
%%sql
update
    ${TABLE_NAME}
set             
    review_body = concat(uuid(),uuid(),uuid(),uuid(),uuid(),uuid(),uuid(),uuid(),uuid(),uuid()),
    timestamp = unix_timestamp(current_timestamp())
where
    review_date = '2003-01-02'
;

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [33]:
%%sh
${WORKSPACE}/hudi-stat.sh s3://${S3_BUCKET}/${TABLE_NAME} timeline commits compactions storage


[ TIMELINE ]

╔═════╤═══════════════════╤═════════════╤═══════════╤═════════════╤═════════════╤═════════════╗
║ No. │ Instant           │ Action      │ State     │ Requested   │ Inflight    │ Completed   ║
║     │                   │             │           │ Time        │ Time        │ Time        ║
╠═════╪═══════════════════╪═════════════╪═══════════╪═════════════╪═════════════╪═════════════╣
║ 0   │ 20230322014803020 │ deltacommit │ COMPLETED │ 03-22 01:48 │ 03-22 01:48 │ 03-22 01:48 ║
╟─────┼───────────────────┼─────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 1   │ 20230322014910700 │ deltacommit │ COMPLETED │ 03-22 01:49 │ 03-22 01:49 │ 03-22 01:49 ║
╟─────┼───────────────────┼─────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 2   │ 20230322014950132 │ deltacommit │ COMPLETED │ 03-22 01:49 │ 03-22 01:49 │ 03-22 01:50 ║
╚═════╧═══════════════════╧═════════════╧═══════════╧═════════════╧═════════════╧═════════════╝

[ COMMITS ]

╔══════════

### 3.8. Offline Schedule

In [34]:
%%sh
# it's required for current user (emr-notebook) to get sudo permission
sudo -u hadoop spark-submit \
  --jars '/usr/lib/hudi/hudi-spark-bundle.jar' \
  --class 'org.apache.hudi.utilities.HoodieCompactor' \
  /usr/lib/hudi/hudi-utilities-bundle.jar \
  --spark-memory '4g' \
  --mode 'schedule' \
  --base-path "s3://${S3_BUCKET}/${TABLE_NAME}" \
  --table-name "$TABLE_NAME" \
  --hoodie-conf "hoodie.compact.inline.max.delta.commits=3" > ${WORKSPACE}/${TABLE_NAME}.schedule.out &>/dev/null

In [35]:
%%sh
${WORKSPACE}/hudi-stat.sh s3://${S3_BUCKET}/${TABLE_NAME} timeline commits compactions storage


[ TIMELINE ]

╔═════╤═══════════════════╤═════════════╤═══════════╤═════════════╤═════════════╤═════════════╗
║ No. │ Instant           │ Action      │ State     │ Requested   │ Inflight    │ Completed   ║
║     │                   │             │           │ Time        │ Time        │ Time        ║
╠═════╪═══════════════════╪═════════════╪═══════════╪═════════════╪═════════════╪═════════════╣
║ 0   │ 20230322014803020 │ deltacommit │ COMPLETED │ 03-22 01:48 │ 03-22 01:48 │ 03-22 01:48 ║
╟─────┼───────────────────┼─────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 1   │ 20230322014910700 │ deltacommit │ COMPLETED │ 03-22 01:49 │ 03-22 01:49 │ 03-22 01:49 ║
╟─────┼───────────────────┼─────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 2   │ 20230322014950132 │ deltacommit │ COMPLETED │ 03-22 01:49 │ 03-22 01:49 │ 03-22 01:50 ║
╟─────┼───────────────────┼─────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 3   │ 20230322015047702

### 3.9. Offline Execute 96.15MB / +1 Compacted File

In [36]:
%%sh
# it's required for current user (emr-notebook) to get sudo permission
sudo -u hadoop spark-submit \
  --jars '/usr/lib/hudi/hudi-spark-bundle.jar' \
  --class "org.apache.hudi.utilities.HoodieCompactor" \
  /usr/lib/hudi/hudi-utilities-bundle.jar \
  --spark-memory '4g' \
  --mode 'execute' \
  --base-path "s3://${S3_BUCKET}/${TABLE_NAME}" \
  --table-name "$TABLE_NAME" > ${WORKSPACE}/${TABLE_NAME}.execute.out &>/dev/null

In [37]:
%%sh
${WORKSPACE}/hudi-stat.sh s3://${S3_BUCKET}/${TABLE_NAME} timeline commits compactions storage


[ TIMELINE ]

╔═════╤═══════════════════╤═════════════╤═══════════╤═════════════╤═════════════╤═════════════╗
║ No. │ Instant           │ Action      │ State     │ Requested   │ Inflight    │ Completed   ║
║     │                   │             │           │ Time        │ Time        │ Time        ║
╠═════╪═══════════════════╪═════════════╪═══════════╪═════════════╪═════════════╪═════════════╣
║ 0   │ 20230322014803020 │ deltacommit │ COMPLETED │ 03-22 01:48 │ 03-22 01:48 │ 03-22 01:48 ║
╟─────┼───────────────────┼─────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 1   │ 20230322014910700 │ deltacommit │ COMPLETED │ 03-22 01:49 │ 03-22 01:49 │ 03-22 01:49 ║
╟─────┼───────────────────┼─────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 2   │ 20230322014950132 │ deltacommit │ COMPLETED │ 03-22 01:49 │ 03-22 01:49 │ 03-22 01:50 ║
╟─────┼───────────────────┼─────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 3   │ 20230322015047702

## 4. Test Case 3 - Async Compaction ( Offline Schedule + Offline Execute )

### 4.1. Test Plan

Step No.|Action|Volume Per Partition |Storage
:--------:|:------|:------|:----------
1|Insert|96MB|+1 Base File
2|Update|804KB|+1 Log File
3|Update|1.2MB|+1 Log File
4|Offline Schedule + Execute|96.15MB|+1 Compacted Base File

### 4.2. Key Settings

KEY|DEFAULT VALUE|SET VALUE
:---|:---|:---
hoodie.compact.inline|false|true
hoodie.compact.schedule.inline|false|false
hoodie.compact.inline.max.delta.commits|5|3
hoodie.copyonwrite.record.size.estimate|1024|175

### 4.3. Set Variables

In [38]:
%%sql
set TABLE_NAME=reviews_mor_compaction_3

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [39]:
%env TABLE_NAME=reviews_mor_compaction_3

env: TABLE_NAME=reviews_mor_compaction_3


### 4.4. Create Table

In [40]:
%%sh
aws s3 rm s3://${S3_BUCKET}/${TABLE_NAME} --recursive &>/dev/null
rm -rf ${WORKSPACE}/${TABLE_NAME}
sleep 5

In [41]:
%%sql
drop table if exists ${TABLE_NAME}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [42]:
%%sql
drop table if exists ${TABLE_NAME}_ro

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [43]:
%%sql
drop table if exists ${TABLE_NAME}_rt

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [44]:
%%sql
create table if not exists ${TABLE_NAME} (
    review_id string, 
    star_rating long, 
    review_body string, 
    review_date date, 
    year long,
    timestamp long,
    parity int
)
using hudi
location 's3://${S3_BUCKET}/${TABLE_NAME}'
partitioned by (parity)
options ( 
    type = 'mor',  
    primaryKey = 'review_id', 
    preCombineField = 'timestamp',
    hoodie.compact.inline = 'false',
    hoodie.compact.schedule.inline = 'false',
    hoodie.compact.inline.max.delta.commits = '3',
    hoodie.copyonwrite.record.size.estimate = '175'
);

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

### 4.5. Insert 96MB / +1 Base File

In [45]:
%%sql
insert into 
    ${TABLE_NAME}
select 
    review_id, 
    star_rating, 
    review_body, 
    review_date, 
    year,
    unix_timestamp(current_timestamp()) as timestamp,
    mod(crc32(review_id), 2) as parity
from
    reviews
where
    year = 2003

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [46]:
%%sh
${WORKSPACE}/hudi-stat.sh s3://${S3_BUCKET}/${TABLE_NAME} timeline commits compactions storage


[ TIMELINE ]

╔═════╤═══════════════════╤═════════════╤═══════════╤═════════════╤═════════════╤═════════════╗
║ No. │ Instant           │ Action      │ State     │ Requested   │ Inflight    │ Completed   ║
║     │                   │             │           │ Time        │ Time        │ Time        ║
╠═════╪═══════════════════╪═════════════╪═══════════╪═════════════╪═════════════╪═════════════╣
║ 0   │ 20230322015250427 │ deltacommit │ COMPLETED │ 03-22 01:52 │ 03-22 01:53 │ 03-22 01:53 ║
╚═════╧═══════════════════╧═════════════╧═══════════╧═════════════╧═════════════╧═════════════╝

[ COMMITS ]

╔═══════════════════╤═════════════════════╤═══════════════════╤═════════════════════╤══════════════════════════╤═══════════════════════╤══════════════════════════════╤══════════════╗
║ CommitTime        │ Total Bytes Written │ Total Files Added │ Total Files Updated │ Total Partitions Written │ Total Records Written │ Total Update Records Written │ Total Errors ║
╠═══════════════════╪════════

### 4.6. Update 787KB / +1 Log File

In [47]:
%%sql
update
    ${TABLE_NAME}
set             
    review_body = concat(uuid(),uuid(),uuid(),uuid(),uuid(),uuid(),uuid(),uuid(),uuid(),uuid()),
    timestamp = unix_timestamp(current_timestamp())
where
    review_date = '2003-01-01'
;

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [48]:
%%sh
${WORKSPACE}/hudi-stat.sh s3://${S3_BUCKET}/${TABLE_NAME} timeline commits compactions storage


[ TIMELINE ]

╔═════╤═══════════════════╤═════════════╤═══════════╤═════════════╤═════════════╤═════════════╗
║ No. │ Instant           │ Action      │ State     │ Requested   │ Inflight    │ Completed   ║
║     │                   │             │           │ Time        │ Time        │ Time        ║
╠═════╪═══════════════════╪═════════════╪═══════════╪═════════════╪═════════════╪═════════════╣
║ 0   │ 20230322015250427 │ deltacommit │ COMPLETED │ 03-22 01:52 │ 03-22 01:53 │ 03-22 01:53 ║
╟─────┼───────────────────┼─────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 1   │ 20230322015414076 │ deltacommit │ COMPLETED │ 03-22 01:54 │ 03-22 01:54 │ 03-22 01:54 ║
╚═════╧═══════════════════╧═════════════╧═══════════╧═════════════╧═════════════╧═════════════╝

[ COMMITS ]

╔═══════════════════╤═════════════════════╤═══════════════════╤═════════════════════╤══════════════════════════╤═══════════════════════╤══════════════════════════════╤══════════════╗
║ CommitTime        

### 4.7. Update 1.2MB / +1 Log File +1 Base File

In [49]:
%%sql
update
    ${TABLE_NAME}
set             
    review_body = concat(uuid(),uuid(),uuid(),uuid(),uuid(),uuid(),uuid(),uuid(),uuid(),uuid()),
    timestamp = unix_timestamp(current_timestamp())
where
    review_date = '2003-01-02'
;

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [50]:
%%sh
${WORKSPACE}/hudi-stat.sh s3://${S3_BUCKET}/${TABLE_NAME} timeline commits compactions storage


[ TIMELINE ]

╔═════╤═══════════════════╤═════════════╤═══════════╤═════════════╤═════════════╤═════════════╗
║ No. │ Instant           │ Action      │ State     │ Requested   │ Inflight    │ Completed   ║
║     │                   │             │           │ Time        │ Time        │ Time        ║
╠═════╪═══════════════════╪═════════════╪═══════════╪═════════════╪═════════════╪═════════════╣
║ 0   │ 20230322015250427 │ deltacommit │ COMPLETED │ 03-22 01:52 │ 03-22 01:53 │ 03-22 01:53 ║
╟─────┼───────────────────┼─────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 1   │ 20230322015414076 │ deltacommit │ COMPLETED │ 03-22 01:54 │ 03-22 01:54 │ 03-22 01:54 ║
╟─────┼───────────────────┼─────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 2   │ 20230322015455449 │ deltacommit │ COMPLETED │ 03-22 01:54 │ 03-22 01:55 │ 03-22 01:55 ║
╚═════╧═══════════════════╧═════════════╧═══════════╧═════════════╧═════════════╧═════════════╝

[ COMMITS ]

╔══════════

### 4.8. Offline Schedule + Execute 96.15MB / +1 Compacted File

In [51]:
%%sh
# it's required for current user (emr-notebook) to get sudo permission
sudo -u hadoop spark-submit \
  --jars '/usr/lib/hudi/hudi-spark-bundle.jar' \
  --class "org.apache.hudi.utilities.HoodieCompactor" \
  /usr/lib/hudi/hudi-utilities-bundle.jar \
  --spark-memory '4g' \
  --mode 'scheduleAndExecute' \
  --base-path "s3://${S3_BUCKET}/${TABLE_NAME}" \
  --table-name "$TABLE_NAME" \
  --hoodie-conf "hoodie.compact.inline.max.delta.commits=3" > ${WORKSPACE}/${TABLE_NAME}.execute.out &>/dev/null

In [52]:
%%sh
${WORKSPACE}/hudi-stat.sh s3://${S3_BUCKET}/${TABLE_NAME} timeline commits compactions storage


[ TIMELINE ]

╔═════╤═══════════════════╤═════════════╤═══════════╤═════════════╤═════════════╤═════════════╗
║ No. │ Instant           │ Action      │ State     │ Requested   │ Inflight    │ Completed   ║
║     │                   │             │           │ Time        │ Time        │ Time        ║
╠═════╪═══════════════════╪═════════════╪═══════════╪═════════════╪═════════════╪═════════════╣
║ 0   │ 20230322015250427 │ deltacommit │ COMPLETED │ 03-22 01:52 │ 03-22 01:53 │ 03-22 01:53 ║
╟─────┼───────────────────┼─────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 1   │ 20230322015414076 │ deltacommit │ COMPLETED │ 03-22 01:54 │ 03-22 01:54 │ 03-22 01:54 ║
╟─────┼───────────────────┼─────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 2   │ 20230322015455449 │ deltacommit │ COMPLETED │ 03-22 01:54 │ 03-22 01:55 │ 03-22 01:55 ║
╟─────┼───────────────────┼─────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 3   │ 20230322015552582

## 5. Test Case 4 - Semi Async Compaction ( Inline Schedule, Offline Execute )

### 5.1. Test Plan

Step No.|Action|Volume Per Partition |Storage
:--------:|:------|:------|:----------
1|Insert|96MB|+1 Base File
2|Update|788KB|+1 Log File
3|Update|1.2MB|+1 Log File
4|Offline Execute|96.15MB|+1 Compacted Base File

### 5.2. Key Settings

KEY|DEFAULT VALUE|SET VALUE
:---|:---|:---
hoodie.compact.inline|false|true
hoodie.compact.schedule.inline|false|false
hoodie.compact.inline.max.delta.commits|5|3
hoodie.copyonwrite.record.size.estimate|1024|175

### 5.3. Set Variables

In [53]:
%%sql
set TABLE_NAME=reviews_mor_compaction_4

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [54]:
%env TABLE_NAME=reviews_mor_compaction_4

env: TABLE_NAME=reviews_mor_compaction_4


### 5.4. Create Table

In [55]:
%%sh
aws s3 rm s3://${S3_BUCKET}/${TABLE_NAME} --recursive &>/dev/null
rm -rf ${WORKSPACE}/${TABLE_NAME}
sleep 5

In [56]:
%%sql
drop table if exists ${TABLE_NAME}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [57]:
%%sql
drop table if exists ${TABLE_NAME}_ro

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [58]:
%%sql
drop table if exists ${TABLE_NAME}_rt

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [59]:
%%sql
create table if not exists ${TABLE_NAME} (
    review_id string, 
    star_rating long, 
    review_body string, 
    review_date date, 
    year long,
    timestamp long,
    parity int
)
using hudi
location 's3://${S3_BUCKET}/${TABLE_NAME}'
partitioned by (parity)
options ( 
    type = 'mor',  
    primaryKey = 'review_id', 
    preCombineField = 'timestamp',
    hoodie.compact.inline = 'false',
    hoodie.compact.schedule.inline = 'true',
    hoodie.compact.inline.max.delta.commits = '3',
    hoodie.copyonwrite.record.size.estimate = '175'
);

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

### 5.5. Insert 96MB / +1 Base File

In [60]:
%%sql
insert into 
    ${TABLE_NAME}
select 
    review_id, 
    star_rating, 
    review_body, 
    review_date, 
    year,
    unix_timestamp(current_timestamp()) as timestamp,
    mod(crc32(review_id), 2) as parity
from
    reviews
where
    year = 2003

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [61]:
%%sh
${WORKSPACE}/hudi-stat.sh s3://${S3_BUCKET}/${TABLE_NAME} timeline commits compactions storage


[ TIMELINE ]

╔═════╤═══════════════════╤═════════════╤═══════════╤═════════════╤═════════════╤═════════════╗
║ No. │ Instant           │ Action      │ State     │ Requested   │ Inflight    │ Completed   ║
║     │                   │             │           │ Time        │ Time        │ Time        ║
╠═════╪═══════════════════╪═════════════╪═══════════╪═════════════╪═════════════╪═════════════╣
║ 0   │ 20230322015709801 │ deltacommit │ COMPLETED │ 03-22 01:57 │ 03-22 01:57 │ 03-22 01:58 ║
╚═════╧═══════════════════╧═════════════╧═══════════╧═════════════╧═════════════╧═════════════╝

[ COMMITS ]

╔═══════════════════╤═════════════════════╤═══════════════════╤═════════════════════╤══════════════════════════╤═══════════════════════╤══════════════════════════════╤══════════════╗
║ CommitTime        │ Total Bytes Written │ Total Files Added │ Total Files Updated │ Total Partitions Written │ Total Records Written │ Total Update Records Written │ Total Errors ║
╠═══════════════════╪════════

### 5.6. Update 787KB / +1 Log File

In [62]:
%%sql
update
    ${TABLE_NAME}
set             
    review_body = concat(uuid(),uuid(),uuid(),uuid(),uuid(),uuid(),uuid(),uuid(),uuid(),uuid()),
    timestamp = unix_timestamp(current_timestamp())
where
    review_date = '2003-01-01'
;

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [63]:
%%sh
${WORKSPACE}/hudi-stat.sh s3://${S3_BUCKET}/${TABLE_NAME} timeline commits compactions storage


[ TIMELINE ]

╔═════╤═══════════════════╤═════════════╤═══════════╤═════════════╤═════════════╤═════════════╗
║ No. │ Instant           │ Action      │ State     │ Requested   │ Inflight    │ Completed   ║
║     │                   │             │           │ Time        │ Time        │ Time        ║
╠═════╪═══════════════════╪═════════════╪═══════════╪═════════════╪═════════════╪═════════════╣
║ 0   │ 20230322015709801 │ deltacommit │ COMPLETED │ 03-22 01:57 │ 03-22 01:57 │ 03-22 01:58 ║
╟─────┼───────────────────┼─────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 1   │ 20230322015834047 │ deltacommit │ COMPLETED │ 03-22 01:58 │ 03-22 01:58 │ 03-22 01:58 ║
╚═════╧═══════════════════╧═════════════╧═══════════╧═════════════╧═════════════╧═════════════╝

[ COMMITS ]

╔═══════════════════╤═════════════════════╤═══════════════════╤═════════════════════╤══════════════════════════╤═══════════════════════╤══════════════════════════════╤══════════════╗
║ CommitTime        

### 5.7. Update 1.2MB / +1 Log File +1 Base File

In [64]:
%%sql
update
    ${TABLE_NAME}
set             
    review_body = concat(uuid(),uuid(),uuid(),uuid(),uuid(),uuid(),uuid(),uuid(),uuid(),uuid()),
    timestamp = unix_timestamp(current_timestamp())
where
    review_date = '2003-01-02'
;

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [65]:
%%sh
${WORKSPACE}/hudi-stat.sh s3://${S3_BUCKET}/${TABLE_NAME} timeline commits compactions storage


[ TIMELINE ]

╔═════╤═══════════════════╤═════════════╤═══════════╤═════════════╤═════════════╤═════════════╗
║ No. │ Instant           │ Action      │ State     │ Requested   │ Inflight    │ Completed   ║
║     │                   │             │           │ Time        │ Time        │ Time        ║
╠═════╪═══════════════════╪═════════════╪═══════════╪═════════════╪═════════════╪═════════════╣
║ 0   │ 20230322015709801 │ deltacommit │ COMPLETED │ 03-22 01:57 │ 03-22 01:57 │ 03-22 01:58 ║
╟─────┼───────────────────┼─────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 1   │ 20230322015834047 │ deltacommit │ COMPLETED │ 03-22 01:58 │ 03-22 01:58 │ 03-22 01:58 ║
╟─────┼───────────────────┼─────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 2   │ 20230322015917952 │ deltacommit │ COMPLETED │ 03-22 01:59 │ 03-22 01:59 │ 03-22 01:59 ║
╟─────┼───────────────────┼─────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 3   │ 20230322015934748

### 5.8. Offline Execute 96.15MB / +1 Compacted File

In [66]:
%%sh
# it's required for current user (emr-notebook) to get sudo permission
sudo -u hadoop spark-submit \
  --jars '/usr/lib/hudi/hudi-spark-bundle.jar' \
  --class "org.apache.hudi.utilities.HoodieCompactor" \
  /usr/lib/hudi/hudi-utilities-bundle.jar \
  --spark-memory '4g' \
  --mode 'execute' \
  --base-path "s3://${S3_BUCKET}/${TABLE_NAME}" \
  --table-name "$TABLE_NAME" > ${WORKSPACE}/${TABLE_NAME}.execute.out &>/dev/null

In [67]:
%%sh
${WORKSPACE}/hudi-stat.sh s3://${S3_BUCKET}/${TABLE_NAME} timeline commits compactions storage


[ TIMELINE ]

╔═════╤═══════════════════╤═════════════╤═══════════╤═════════════╤═════════════╤═════════════╗
║ No. │ Instant           │ Action      │ State     │ Requested   │ Inflight    │ Completed   ║
║     │                   │             │           │ Time        │ Time        │ Time        ║
╠═════╪═══════════════════╪═════════════╪═══════════╪═════════════╪═════════════╪═════════════╣
║ 0   │ 20230322015709801 │ deltacommit │ COMPLETED │ 03-22 01:57 │ 03-22 01:57 │ 03-22 01:58 ║
╟─────┼───────────────────┼─────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 1   │ 20230322015834047 │ deltacommit │ COMPLETED │ 03-22 01:58 │ 03-22 01:58 │ 03-22 01:58 ║
╟─────┼───────────────────┼─────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 2   │ 20230322015917952 │ deltacommit │ COMPLETED │ 03-22 01:59 │ 03-22 01:59 │ 03-22 01:59 ║
╟─────┼───────────────────┼─────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 3   │ 20230322015934748